In [7]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
import uuid
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
#from gensim import corpora, models
from nltk import pos_tag
#from gensim.models import CoherenceModel


from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
import re
#import spacy
#import en_core_web_sm
from pandas.core.common import flatten
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
#from gensim.models import Word2Vec
  
lemmatizer = WordNetLemmatizer() 



In [8]:
# Importing and adding stop words

nltk.download('stopwords')
stop = stopwords.words('english')
stop_add = ['said', 'year', 'like', 'says', 'years', 'month', 'months', 'day', 'days', 'say', 'would', 'in', 'of', '.', ',',':',';','`',"''",'s','-']
for new in stop_add:
    stop.append(new)
print(stop)



['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gowthamasokan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Function to remove stopwords

def remove_stopwords(text):
    stop_words = stopwords.words('english')
    stop_words.extend(["also", "and", "its", "bring", "got", "from", "lot", "rather", "even", "from", "but","that","in","you","who","to", "of", "become", "thats", "looking"])
    words = [w for w in text if w not in stop_words]
    return words



In [11]:
# Loading the text

Syllabi = pd.read_excel("Syllabi collection - Gowtha,.xlsx")
Syllabi

,University,Course,Link,Text,Notes
0,Arizona State University,Principles of Public Interest Technology,https://webapp4.asu.edu/bookstore/viewsyllabus...,PIT 501: Principles of Public Interest Techno...,NaN
1,Boston University,XCC433: Justice Media co-Lab,https://www.bu.edu/cds-faculty/files/2021/04/C...,THE JUSTICE MEDIA CO-LAB Journalism is strong...,NaN
2,Boston University,Law & Algorithms,https://cs-people.bu.edu/kaptchuk/teaching/ds4...,Page 1 of 8 Law and Algorithms – Spring 2022 ...,NaN
3,Boston University,Data Science for Politics,NaN,NaN,"Need link, ask David Palmer"
4,Cal Poly State,The art of ethical design,https://express.adobe.com/page/tLOTJ1tjgToa8/,View Static Version Course Principles. Student...,NaN
5,Cal Poly State,Humanity for Sale: Ethics in Business and Tech...,https://static1.squarespace.com/static/5e6488a...,Humanity for sale Ethics in Business and Tech...,NaN
6,Cal Poly State,Modern Ethical Technology,https://static1.squarespace.com/static/5e6488a...,ENGR 380: Modern Ethical Technology Kevin Dix...,NaN
7,Cal Poly State,Ethical tech in psychologcy,https://static1.squarespace.com/static/5e6488a...,Macy Gordon Maya Pulfer Candace Cabrera Nicol...,NaN
8,Cal Poly State,Considerations before code implementation,https://static1.squarespace.com/static/5e6488a...,"Jordan Klein, Melina Lopez, Ana Marsh, and Av...",NaN
9,Cal Poly State,Ethical technologies in animal science,https://static1.squarespace.com/static/5e6488a...,1 Animal Science 380: Ethical Technologies in...,NaN


In [12]:
# Example text

Syllabi['Text'].iloc[0]

' PIT 501: Principles of Public Interest Technology Professor: Erik Fisher Zoom office: https://asu.zoom.us/j/7712166804 efisher1@asu.edu Office hours: 3-4 Tuesdays and as needed. Please email me to schedule Zoom office appointments. Course Description This course introduces students to the fundamentals of public interest technology (PIT), including key concepts, theories and frameworks. Students will identify and analyze stakeholders, societal dimensions, and policy issues in the context of current and emerging technologies such as facial recognition technology, biotechnology, artificial intelligence and financial technology. Students will be introduced to the importance of socio-technical change, public values, anticipatory governance,1 of 8 **Disclaimer** This syllabus is to be used as a guideline only. The information provided is a summary of topics to be covered in the class. Information contained in this document such as assignments, grading scales, due dates, office hours, requi

In [13]:
# Example text

Syllabi['Text'].iloc[1]

' THE JUSTICE MEDIA CO-LAB Journalism is stronger when informed by and infused with data, which provide information about patterns that may be otherwise invisible. But many journalists lack access to “big data”—and the skillsets to interpret and use it compellingly. Enter Boston University’s Justice Media co-Lab, which supports the creation of excellent, data- driven journalism—and prepares BU students to leverage the power of computing and data sciences to advance justice and transparency in their future careers. A partnership between BU’s College of Communication (COM) and the new Faculty of Computing & Data Sciences (CDS), the Justice Media co-Lab engages faculty, students, and external experts in research and innovation collaborations and curricular and co-curricular programs with the potential for real-world impact. The theme of “justice” is interpreted broadly to allow for the exploration of a wide range of intersectional topics, from racial justice to environmental justice to is

In [ ]:
# Calling the column name from the dataframe
# This should be the course description column

df = df['column_name']

stop_clean = remove_stopwords(df)

str_list = list(map(str, stop_clean)) # Converting all datatypes to strings
#porter = PorterStemmer()

lemmatized = []
for s in str_list:
    lemmatized.append(lemmatizer.lemmatize(s))



tokens = []
for i in lemmatized:
    i.lower()
    tokens.append(word_tokenize(i))
    

#Word2Vec model to make the word embeddings
#model = Word2Vec(tokens, min_count=1, size=50, workers=3, window=2, sg=1)

